In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat

# from extract_graph import dic_to_sparse
from util.sys import get_path, shift_skeleton
from plotutil import (
    show_im,
    overlap,
    show_im_rgb,
    plot_nodes,
    plot_nodes_from_list,
    plot_t_tp1,
)
from extract_graph import (
    generate_graph_tab_from_skeleton,
    generate_nx_graph_from_skeleton,
    generate_skeleton,
    clean,
)
import networkx as nx
from node_id import (
    second_identification,
    whole_movement_identification,
    first_identification,
    relabel_nodes,
    clean_nodes,
    orient,
)
from extract_graph import (
    dic_to_sparse,
    from_sparse_to_graph,
    generate_nx_graph,
    prune_graph,
    from_nx_to_tab,
    from_nx_to_tab_matlab,
    sparse_to_doc,
    connections_pixel_list_to_tab,
    transform_list,
)
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util.sys import get_path
import pandas as pd
from datetime import datetime, timedelta
import ast
from time import time
import os
from random import choice
from pycpd import RigidRegistration, DeformableRegistration
from cycpd import rigid_registration

In [17]:
plt.close("all")
X = []
dimx = 1000
dimy = 1000
for i in range(dimx):
    X.append((i, 0))
    X.append((i, dimy))
for i in range(dimy):
    X.append((dimx, i))
    X.append((0, i))
X = np.transpose(np.array(X))
angle = np.random.uniform(0, np.pi / 2)
R = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
t = np.random.uniform(0, 1000, (2, 1))
print(R, t)
np.random.shuffle(np.transpose(X))
Y = np.transpose(np.transpose(np.dot(R, X)) + np.transpose(t)) + np.random.uniform(
    0, dimx / 10, (2, 4 * dimx)
)
np.random.shuffle(np.transpose(Y))
Y = Y[:, : dimx // 4]
X = X[:, : dimx // 8]
fig = plt.figure(figsize=(10, 9))

ax = fig.add_subplot(111)
ax.scatter(X[0, :], X[1, :])
ax.scatter(Y[0, :], Y[1, :])

X.shape, Y.shape

[[ 0.83060772 -0.55685799]
 [ 0.55685799  0.83060772]] [[187.76557008]
 [529.49248594]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((2, 125), (2, 250))

In [18]:
reg = rigid_registration(
    **{
        "X": np.transpose(X.astype(float)),
        "Y": np.transpose(Y.astype(float)),
        "scale": False,
    }
)
out = reg.register()
Rfound = reg.R[0:2, 0:2]
tfound = np.dot(Rfound, reg.t[0:2])

Iteration:1
ML:  1074.430; 	ML change (error):  1074.430; 	Sigma^2:232138.366; 	Sigma^2 change:369215.420
[                                                                        ]
Iteration:2
ML:   979.311; 	ML change (error):    95.119; 	Sigma^2:145109.691; 	Sigma^2 change: 87028.675
[=                                                                       ]
Iteration:3
ML:   954.853; 	ML change (error):    24.458; 	Sigma^2: 94188.562; 	Sigma^2 change: 50921.129
[==                                                                      ]
Iteration:4
ML:   934.021; 	ML change (error):    20.832; 	Sigma^2: 59339.295; 	Sigma^2 change: 34849.267
[==                                                                      ]
Iteration:5
ML:   911.389; 	ML change (error):    22.632; 	Sigma^2: 37000.180; 	Sigma^2 change: 22339.116
[===                                                                     ]
Iteration:6
ML:   889.029; 	ML change (error):    22.360; 	Sigma^2: 23940.575; 	Sigma^2 change:

In [86]:
Rfound, tfound, R, t

(array([[ 0.95229604,  0.30517577],
        [-0.30517577,  0.95229604]]),
 array([ 895.62959999, -364.20998122]),
 array([[ 0.30495782, -0.95236586],
        [ 0.95236586,  0.30495782]]),
 array([[47.7737877],
        [49.2362971]]))

In [19]:
fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111)
Yrep = np.transpose(
    np.transpose(np.dot(Rfound, np.transpose(np.transpose(X)))) - tfound
)
ax.scatter(np.transpose(Y)[:, 0], np.transpose(Y)[:, 1])
ax.scatter(np.transpose(Yrep)[:, 0], np.transpose(Yrep)[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
reg = RigidRegistration(**{"X": np.transpose(X), "Y": np.transpose(Y)})
out = reg.register()
Rfound = reg.R
Yrep = np.dot(Rfound, np.transpose(np.transpose(X)))
reg = RigidRegistration(**{"X": np.transpose(Yrep), "Y": np.transpose(Y)})
out = reg.register()
tfound = reg.t

In [52]:
def try_dim(dim):
    X = []
    dimx = dim
    dimy = dim
    for i in range(dimx):
        X.append((i, 0))
        X.append((i, dimy))
    for i in range(dimy):
        X.append((dimx, i))
        X.append((0, i))
    X = np.transpose(np.array(X))
    angle = np.random.uniform(0, np.pi / 2)
    R = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    t = np.random.uniform(0, 100, (2, 1))
    print(R, t)
    Y = np.transpose(np.transpose(np.dot(R, X)) + np.transpose(t)) + np.random.uniform(
        0, 5, (2, 4 * dimx)
    )
    np.random.shuffle(np.transpose(Y))
    reg = RigidRegistration(**{"X": np.transpose(X), "Y": np.transpose(Y)})
    out = reg.register()
    Rfound = reg.R
    Yrep = np.dot(Rfound, np.transpose(np.transpose(X)))
    reg = RigidRegistration(**{"X": np.transpose(Yrep), "Y": np.transpose(Y)})
    out = reg.register()
    tfound = reg.t

In [ ]:
import cProfile

cProfile.run("try_dim(2500)")

In [5]:
plate = 27
directory = (
    "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
)
listdir = os.listdir(directory)
list_dir_interest = [
    name
    for name in listdir
    if name.split("_")[-1] == f'Plate{0 if plate<10 else ""}{plate}'
]
ss = [name.split("_")[0] for name in list_dir_interest]
ff = [name.split("_")[1] for name in list_dir_interest]
dates_datetime = [
    datetime(
        year=int(ss[i][:4]),
        month=int(ss[i][4:6]),
        day=int(ss[i][6:8]),
        hour=int(ff[i][0:2]),
        minute=int(ff[i][2:4]),
    )
    for i in range(len(list_dir_interest))
]
dates_datetime.sort()
dates_datetime_chosen = dates_datetime[1:6]
dates = [
    f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}'
    for date in dates_datetime_chosen
]
mat_skels = [read_mat(get_path(date, plate, True))["skel"] for date in dates]
dic_skels = [dic_to_sparse(mat_skel) for mat_skel in mat_skels]
skeleton_docs = [sparse_to_doc(sparse_skel) for sparse_skel in dic_skels]

In [23]:
dates_datetime_chosen

[datetime.datetime(2020, 7, 13, 11, 44),
 datetime.datetime(2020, 7, 13, 15, 44),
 datetime.datetime(2020, 7, 13, 19, 44),
 datetime.datetime(2020, 7, 13, 23, 44),
 datetime.datetime(2020, 7, 14, 3, 44)]

In [8]:
non_zeros = skeleton_docs[1].keys()

In [9]:
def get_neighbours(pixel, non_zero_pixel):
    x = pixel[0]
    y = pixel[1]
    primary_neighbours = {(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)}
    secondary_neighbours = {
        (x + 1, y - 1),
        (x + 1, y + 1),
        (x - 1, y + 1),
        (x - 1, y - 1),
    }
    num_neighbours = 0
    actual_neighbours = []
    for neighbour in primary_neighbours:
        if neighbour in non_zero_pixel:
            num_neighbours += 1
            xp = neighbour[0]
            yp = neighbour[1]
            primary_neighboursp = {
                (xp + 1, yp),
                (xp - 1, yp),
                (xp, yp + 1),
                (xp, yp - 1),
            }
            for neighbourp in primary_neighboursp:
                secondary_neighbours.discard(neighbourp)
            actual_neighbours.append(neighbour)
    for neighbour in secondary_neighbours:
        if neighbour in non_zero_pixel:
            num_neighbours += 1
            actual_neighbours.append(neighbour)
    return (actual_neighbours, num_neighbours)


def get_degree3_nodes(skel):
    deg_3 = []
    non_zero = skel.keys()
    for pixel in non_zero:
        n, num = get_neighbours(pixel, non_zero)
        if num == 3:
            deg_3.append(pixel)
    return deg_3

In [20]:
node_center = choice(deg_3)

In [21]:
node_center

(11337, 13152)

In [11]:
deg_3 = get_degree3_nodes(skeleton_docs[1])

In [22]:
plt.close("all")
fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111)
window = 1000
ax.imshow(
    dic_skels[1][
        node_center[0] - window : node_center[0] + window,
        node_center[1] - window : node_center[1] + window,
    ].todense()
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
def find_common_group_nodes(
    Sa, Sb, degree3_nodesa, degree3_nodesb, posa, posb, R0, t0, window=500, maxdist=50
):
    common_nodes_a = []
    common_nodes_b = []
    common_centroida = []
    common_centroidb = []
    t = time()
    posarottrans = {
        key: np.round(
            np.transpose(np.dot(R0, np.transpose(np.array(posa[key]))) + t0)
        ).astype(np.int)
        for key in degree3_nodesa
    }
    print("rotating translating", time() - t)
    for node in degree3_nodesa:
        t = time()
        posanchor = posarottrans[node]
        potential_surroundinga = Sa[
            posanchor[0] - 2 * window : posanchor[0] + 2 * window,
            posanchor[1] - 2 * window : posanchor[1] + 2 * window,
        ]
        potential_surroundingb = Sb[
            posanchor[0] - 2 * window : posanchor[0] + 2 * window,
            posanchor[1] - 2 * window : posanchor[1] + 2 * window,
        ]
        #         print("candidates",len(potential_surroundinga.data))
        #         print("finding_potential_surrounding",time()-t)
        t = time()
        surrounding_nodesa = [
            node
            for node in potential_surroundinga.data
            if (
                posanchor[0] - window
                < posarottrans[int(node)][0]
                < posanchor[0] + window
                and posanchor[1] - window
                < posarottrans[int(node)][1]
                < posanchor[1] + window
            )
        ]
        surrounding_nodesb = [
            node
            for node in potential_surroundingb.data
            if (
                posanchor[0] - window < posb[int(node)][0] < posanchor[0] + window
                and posanchor[1] - window < posb[int(node)][1] < posanchor[1] + window
            )
        ]
        #         print("finding_surrounding",time()-t)
        t = time()
        if len(surrounding_nodesa) == len(surrounding_nodesb):
            possurroundinga = [posarottrans[node] for node in surrounding_nodesa]
            possurroundingb = [posb[node] for node in surrounding_nodesb]
            centroida = np.mean(possurroundinga, axis=0)
            centroidb = np.mean(possurroundingb, axis=0)
            if np.linalg.norm(centroida - centroidb) <= maxdist:
                common_centroida.append(centroida)
                common_centroidb.append(centroidb)
    return (common_centroida, common_centroidb)


def realign2(
    skeleton1, skeleton2, convergence_threshold, window=500, maxdist=50, save=""
):
    converged = False
    tim = time()
    nx_graphA, posA = generate_nx_graph_from_skeleton(skeleton1)
    nx_graphB, posB = generate_nx_graph_from_skeleton(skeleton2)
    print("generate_nx_graph_from_skeleton, t=", tim - time())
    tim = time()
    t0 = np.array([0, 0])
    R0 = np.identity(2)
    degree3_nodesa = [node for node in nx_graphA if nx_graphA.degree(node) == 3]
    degree3_nodesb = [node for node in nx_graphB if nx_graphB.degree(node) == 3]
    print("lennodes=", len(degree3_nodesa))
    Sa = sparse.csr_matrix((22000, 46000))
    Sb = sparse.csr_matrix((22000, 46000))
    for node in degree3_nodesa:
        Sa[posA[node][0], posA[node][1]] = node
    for node in degree3_nodesb:
        Sb[posB[node][0], posB[node][1]] = node
    while not converged:
        listeA, listeB = find_common_group_nodes(
            Sa,
            Sb,
            degree3_nodesa,
            degree3_nodesb,
            posA,
            posB,
            R0,
            t0,
            maxdist=maxdist,
            window=window,
        )
        H = np.dot(
            np.transpose(np.array(listeA) - np.mean(listeA, axis=0)),
            np.array(listeB) - np.mean(listeB, axis=0),
        )
        U, S, V = np.linalg.svd(H)
        R = np.dot(V, np.transpose(U))
        t = np.mean(listeB, axis=0) - np.dot(R, np.mean(listeA, axis=0))
        print("number_common_nodes_found :", len(listeA))
        if np.linalg.norm(t) <= convergence_threshold:
            converged = True
        R0 = np.dot(R, R0)
        t0 = t + t0
    print("Find R and T, t=", tim - time())
    tim = time()
    skeleton_transformed = transform_skeleton(skeleton1, R0, t0)
    skeleton_transformed = dilate(skeleton_transformed)
    skeleton_transformed = zhangSuen(skeleton_transformed)
    print("transform, dilate and thin, t=", tim - time())
    tim = time()
    if len(save) >= 0:
        from_nx_to_tab(*generate_nx_graph_from_skeleton(skeleton_transformed)).to_csv(
            save + "_raw_aligned_skeleton.csv"
        )
        np.savetxt(save + "rot.txt", R0)
        np.savetxt(save + "trans.txt", t0)
    print("R0=", R0, "t0=", t0)
    return skeleton_transformed

In [39]:
def transform_skeleton(skeleton_doc, Rot, trans):
    transformed_skeleton = {}
    transformed_keys = np.round(
        np.transpose(np.dot(Rot, np.transpose(np.array(list(skeleton_doc.keys())))))
        + trans
    ).astype(np.int)
    i = 0
    for pixel in list(transformed_keys):
        i += 1
        transformed_skeleton[(pixel[0], pixel[1])] = 1
    return transformed_skeleton

In [31]:
S = sparse.csr_matrix((22000, 46000))
S[12000:14000, 12000:14000] = 1

In [34]:
S.data

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
import cProfile

cProfile.run("S[12000:14000,12000:14000]")

In [40]:
r = realign2(skeleton_docs[4], skeleton_docs[3], 2, save="Data/", maxdist=70)

generate_nx_graph_from_skeleton, t= -586.73597240448
lennodes= 13604
rotating translating 0.16901159286499023
number_common_nodes_found : 328
rotating translating 0.2030162811279297
number_common_nodes_found : 402
rotating translating 0.18901276588439941
number_common_nodes_found : 480
rotating translating 0.1980135440826416
number_common_nodes_found : 635
rotating translating 0.18399858474731445
number_common_nodes_found : 743
rotating translating 0.1960141658782959
number_common_nodes_found : 834
rotating translating 0.1940152645111084
number_common_nodes_found : 832
rotating translating 0.20001506805419922
number_common_nodes_found : 872
rotating translating 0.19201445579528809
number_common_nodes_found : 904
Find R and T, t= -203.0311245918274
transform, dilate and thin, t= -374.64126110076904
R0= [[ 0.99999879  0.00155542]
 [-0.00155542  0.99999879]] t0= [ -2.50883403 -72.90254211]


In [24]:
r0 = realign(skeleton_docs[4], nx_graph_pivot, pos_pivot, 2, save="Data/", maxdist=150)

number_common_nodes_found : 356
number_common_nodes_found : 465
number_common_nodes_found : 489
992004 993235
995666 995743
133466 3100
1376 924
606 475
410 375
306 273
216 196
169 125
122 105
90 69
72 58
60 45
48 36
38 27
27 21
22 16
15 11
9 6
3 0
0 0
R0= [[ 9.99999527e-01 -9.72716200e-04]
 [ 9.72716200e-04  9.99999527e-01]] t0= [42.25170203  6.08798379]


In [25]:
r1 = realign(skeleton_docs[4], nx_graph_pivot, pos_pivot, 2, save="Data/", maxdist=50)

number_common_nodes_found : 290
number_common_nodes_found : 435
number_common_nodes_found : 454
number_common_nodes_found : 465
992012 993234
995675 995706
133613 3119
1377 925
612 475
410 375
306 273
216 196
169 125
122 105
90 69
72 58
60 45
48 36
38 27
27 21
22 16
15 11
9 6
3 0
0 0
R0= [[ 0.99999924 -0.00123398]
 [ 0.00123398  0.99999924]] t0= [47.7829256   3.93269705]


In [27]:
def make_sparse(dico):
    dim = (20800, 46000)
    skel = sparse.dok_matrix(dim, dtype=bool)
    for key in dico.keys():
        skel[key] = dico[key]
    return skel

In [31]:
skeletons = [make_sparse(skeleton_docs[0]), make_sparse(r0), make_sparse(r1)]

In [37]:
from plotutil import plot_t_tp1, compress_skeleton

factor = 5
final_pictures = [
    compress_skeleton(skeletons[i], factor) >= 1 for i in range(len(skeletons))
]

In [43]:
plot_t_tp1(
    [],
    [],
    None,
    None,
    final_pictures[0],
    final_pictures[2],
    compress=5,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
plt.close("all")

In [22]:
r = realign(skeleton_docs[4], nx_graph_pivot, pos_pivot, 2, save="Data/", maxdist=30)

number_common_nodes_found : 55
number_common_nodes_found : 170
number_common_nodes_found : 417
number_common_nodes_found : 453
991937 993213
995616 995688
133668 3119
1389 929
613 475
410 375
306 273
216 196
169 125
122 105
88 67
72 58
60 45
48 36
38 27
27 21
22 16
15 11
9 6
3 0
0 0
R0= [[ 0.99999923 -0.00124197]
 [ 0.00124197  0.99999923]] t0= [48.09796892  3.71029006]


In [23]:
r = realign(skeleton_docs[4], nx_graph_pivot, pos_pivot, 2, save="Data/", maxdist=40)

number_common_nodes_found : 209
number_common_nodes_found : 405
number_common_nodes_found : 450
number_common_nodes_found : 458
991983 993211
995655 995687
133685 3099
1373 930
619 476
410 375
306 273
216 196
169 125
122 105
90 69
72 58
60 45
48 36
38 27
27 21
22 16
15 11
9 6
3 0
0 0
R0= [[ 0.99999925 -0.00122743]
 [ 0.00122743  0.99999925]] t0= [47.84393103  4.08633907]
